<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/TF_LSTM_Litemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly --upgrade

     |████████████████████████████████| 517.2MB 30kB/s 
     |████████████████████████████████| 460kB 48.8MB/s 
     |████████████████████████████████| 2.9MB 38.7MB/s 


In [0]:
import numpy as np
import tensorflow as tf

In [3]:
# Step 1: Build the MNIST LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28), name='input'),
    tf.keras.layers.LSTM(20),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20)                3920      
_________________________________________________________________
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
output (Dense)               (None, 10)                210       
Total params: 4,130
Trainable params: 4,130
Non-trainable params: 0
_________________________________________________________________


In [4]:
#2 Step 2: Train & Evaluate the model
# Load MNIST dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# Change this to True if you want to test the flow rapidly.
# Train with a small dataset and only 1 epoch. The model will work poorly
# but this provides a fast way to test if the conversion works end to end.
_FAST_TRAINING = False
_EPOCHS = 5
if _FAST_TRAINING:
  _EPOCHS = 1
  _TRAINING_DATA_COUNT = 1000
  x_train = x_train[:_TRAINING_DATA_COUNT]
  y_train = y_train[:_TRAINING_DATA_COUNT]

model.fit(x_train, y_train, epochs=_EPOCHS)
model.evaluate(x_test, y_test, verbose=0)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.8305 - accuracy: 0.7394
Epoch 2/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.3034 - accuracy: 0.9111
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2066 - accuracy: 0.9396
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1624 - accuracy: 0.9527
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1375 - accuracy: 0.9597


[0.14083439111709595, 0.9570000171661377]

In [0]:
#Step 3: Convert the Keras model to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Note: It will NOT work without enabling the experimental converter!
# `experimental_new_converter` flag.
converter.experimental_new_converter = True
tflite_model = converter.convert()

In [0]:
import pathlib
tflite_models_dir = pathlib.Path("./lstm_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [8]:
tflite_model_file = tflite_models_dir/"lstm_model.tflite"
tflite_model_file.write_bytes(tflite_model)

28756

In [9]:
# Post training quantization
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"lstm_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

18672

In [10]:
ls -lh {tflite_models_dir}

total 52K
-rw-r--r-- 1 root root 19K Apr  7 17:17 lstm_model_quant.tflite
-rw-r--r-- 1 root root 29K Apr  7 17:16 lstm_model.tflite


In [0]:
from google.colab import files

In [0]:
files.download(tflite_model_quant_file)

In [0]:
files.download(tflite_model_file)